In [21]:
%pip install -U langgraph langchain-tavily langgraph-checkpoint-sqlite

In [22]:
!pip install openai

In [23]:
!pip install -U jupyter-client

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.1/106.1 kB 2.6 MB/s eta 0:00:00
  Attempting uninstall: jupyter-client
    Found existing installation: jupyter-client 6.1.12
    Uninstalling jupyter-client-6.1.12:
      Successfully uninstalled jupyter-client-6.1.12
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
notebook 6.5.7 requires jupyter-client<8,>=5.3.4, but you have jupyter-client 8.6.3 which is incompatible.
jupyter-kernel-gateway 2.5.2 requires jupyter-client<8.0,>=5.2.0, but you have jupyter-client 8.6.3 which is incompatible.


In [24]:
!pip install -U langchain-openai

In [25]:
from langgraph.prebuilt import create_react_agent

In [26]:
from langchain_core.tools import tool

In [27]:
pip install langchain pydantic requests


In [28]:
import os
import time
import re
import requests
from urllib.parse import urlparse, parse_qsl, urlencode, urlunparse
from typing import List, Optional, Dict, Any
from pydantic import BaseModel, Field
from google.colab import userdata
from langchain.tools import StructuredTool

GOOGLE_CSE_API_KEY = userdata.get("GOOGLE_CSE")
GOOGLE_CSE_CX = userdata.get("GOOGLE_CSE_CX")
GOOGLE_CSE_ENDPOINT = "https://www.googleapis.com/customsearch/v1"



#def now_iso() -> str:
    #return time.strftime("%Y-%m-%dT%H:%M:%SZ", time.gmtime())

def canonicalize_url(url: str) -> str:
    try:
        parts = urlparse(url)
        netloc = parts.netloc.lower().replace(":80","").replace(":443","")
        q = [(k,v) for k,v in parse_qsl(parts.query) if not k.startswith("utm_") and k not in {"gclid","fbclid"}]
        return urlunparse((parts.scheme, netloc, parts.path, parts.params, urlencode(q), parts.fragment))
    except Exception:
        return url

class GoogleSearchInput(BaseModel):
    query: str = Field(..., description="Web search query")
    k: int = Field(5, ge=1, le=10, description="Max results to return")
    safe: str = Field("active", description="SafeSearch mode: active|off")
    lr: Optional[str] = Field(None, description="Language restrict, e.g. 'lang_en'")

def google_search(query: str, k: int = 5, safe: str = "active", lr: Optional[str] = None) -> List[Dict[str, Any]]:
    if not GOOGLE_CSE_API_KEY or not GOOGLE_CSE_CX:
        raise RuntimeError("Set GOOGLE_CSE_API_KEY and GOOGLE_CSE_CX env vars")

    params = {
        "key": GOOGLE_CSE_API_KEY,
        "cx": GOOGLE_CSE_CX,
        "q": query,
        "num": min(k, 10),
        "safe": safe
    }
    if lr:
        params["lr"] = lr #its optional.

    resp = requests.get(GOOGLE_CSE_ENDPOINT, params=params, timeout=15)
    resp.raise_for_status()
    data = resp.json()

    hits = []
    for it in data.get("items", []):
        title = (it.get("title") or "").strip()
        url = canonicalize_url((it.get("link") or "").strip())
        snippet = (it.get("snippet") or "").strip()
        if not title or not url:
            continue
        snippet = re.sub(r"\s+", " ", snippet)
        hits.append({
            "title": title,
            "url": url,
            "snippet": snippet,
            "source": "google_cse"
            #"seen_at": now_iso()
        })
    return hits[:k]

GoogleSearchTool = StructuredTool.from_function(
    func=google_search,
    name="web_search",
    description="Search return {title,url,snippet,source,seen_at}.",
    args_schema=GoogleSearchInput
)


In [35]:
from langchain_openai import ChatOpenAI
from langchain.agents import initialize_agent, AgentType
from langgraph.prebuilt import create_react_agent
from google.colab import userdata
import openai

user_input = input("What is your article about?")

perspectives = ['Molecular Biologist', 'Medical Researcher', 'Agricultural Scientist', 'Bioethicist', 'Regulatory Policy Expert', 'Basic Fact Researcher']

LLM_1 = ChatOpenAI(model="gpt-4o-mini", openai_api_key=userdata.get("OPENAI_API_KEY"))
LLM_2 = ChatOpenAI(model="gpt-4.1", openai_api_key=userdata.get("OPENAI_API_KEY"))

tools = [GoogleSearchTool]
EXPERT_RESEARCHER = create_react_agent(LLM_2, tools)

convo = []
M = 5
for p in perspectives:
  convo_history = []
  for i in range(M):
    query = f"You are a Wikipedia Writer and you are writing an article on {user_input}, ask a question from the perspective of a {p} to contribute to further understanding of the topic based on the given conversation history you have with an expert: {convo_history}. Only respond with the question"
    q = LLM_1.invoke([{"role": "user", "content": query}])
    convo_history.append(q.content) # Use .content to get the response string
    input_message = {"role": "user", "content": f"Split the given query: {q} into Google searches in order to best answer the question, respond with at most 3 credible sources from the web in addition to your answer which you synthesized from the searched information"}
    response = EXPERT_RESEARCHER.invoke({"messages": [input_message]})
    print(response)
    for message in response["messages"]:
      message.pretty_print()

  convo.append(convo_history)

print("end")

What is your article about?CRISPR
{'messages': [HumanMessage(content="Split the given query: content='What are the potential off-target effects of CRISPR-Cas9 gene editing, and how can we improve the specificity of this technology to minimize unintended genetic alterations?' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 33, 'prompt_tokens': 60, 'total_tokens': 93, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'id': 'chatcmpl-C8BSkZlWG1Lq2D82WN2u3bPYezkvW', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None} id='run--de7b932b-b432-476d-8dff-0f5fb1327df0-0' usage_metadata={'input_tokens': 60, 'output_tokens': 33, 'total_tokens': 93, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'out

In [41]:
openai.api_key = userdata.get("OPENAI_API_KEY")
response = openai.chat.completions.create(
model="gpt-4o-mini",
messages=[
      {"role": "system", "content": "You are a Wikipedia Article Writer and Editor"},
       {"role": "user", "content": f"Generate an outline of a hypothetical wikipedia article on the topic: {user_input}, * is for headings, ** is for subheadings and *** is for sub-subheadings if necessary. Only respond with the outline and nothing else"},
      ]
)

outline_draft = response.choices[0].message.content


outline = generate_article_outline(convo, outline_draft, user_input)
print(outline)

# CRISPR

## Introduction
### Definition of CRISPR
### Historical Background
### Importance in Genetic Research

## CRISPR-Cas9 Mechanism
### Overview of CRISPR-Cas9 Mechanism
### Components of CRISPR-Cas9 System
#### CRISPR sequences
#### Cas proteins
### Gene-editing Process
#### Key Steps in the CRISPR-Cas9 Process
#### Differences from Traditional Genetic Engineering Techniques
### Enhancing Specificity of CRISPR-Cas9 System
#### Off-Target Effects and Their Mitigation
#### Improving Specificity of Genetic Alterations

## Applications of CRISPR
### Medical Applications
#### Gene Therapy
#### Treatment of Genetic Disorders
#### Risks in Human Therapeutic Applications
### Agricultural Applications
#### Crop Improvement
#### Pest Resistance
#### Enhancing Crop Resilience to Climate Change-Related Stresses
### Industrial Applications
#### Biofuels
#### Bioremediation

## Ethical and Ecological Considerations
### Ethical Debates in Gene Editing
### Potential Ecological Consequences
### 

In [39]:
def generate_article_outline(convo, outline_draft, topic):
  #topic is always user_input
  response = openai.chat.completions.create(
  model="gpt-4",
  messages=[
      {"role": "system", "content": "You are a Wikipedia Article Writer and Editor"},
       {"role": "user", "content": f"Given a topic, an outline draft, and a conversation on the given topic, generate a comprehensive wikipedia article outline. The outline consists of headings (#), subheadings (##) and occaisonally sub subheadings (###) only respond with the outline and nothing else. topic: {topic}, outline draft: {outline_draft}, conversation: {convo}"},
      ]
  )
  return response.choices[0].message.content

In [42]:
def final_article(user_input, sources, outline):
  sections = """ * CRISPR
              ** Introduction
              *** Definition of CRISPR
              *** Historical Background
              *** Importance in Genetic Research


              ** CRISPR-Cas9 Mechanism
              *** Overview of CRISPR-Cas9 Mechanism
              *** Components of CRISPR-Cas9 System
              **** CRISPR sequences
              **** Cas proteins
              *** Gene-editing Process
              **** Key Steps in the CRISPR-Cas9 Process
              **** Differences from Traditional Genetic Engineering Techniques
              *** Enhancing Specificity of CRISPR-Cas9 System
              **** Off-Target Effects and Their Mitigation
              **** Improving Specificity of Genetic Alterations


              ** Applications of CRISPR
              *** Medical Applications
              **** Gene Therapy
              **** Treatment of Genetic Disorders
              **** Risks in Human Therapeutic Applications
              *** Agricultural Applications
              **** Crop Improvement
              **** Pest Resistance
              **** Enhancing Crop Resilience to Climate Change-Related Stresses
              *** Industrial Applications
              **** Biofuels
              **** Bioremediation


              ** Ethical and Ecological Considerations
              *** Ethical Debates in Gene Editing
              *** Potential Ecological Consequences
              *** Potential Risks and Concerns
              **** Off-Target Effects
              **** Germline Editing


              ** Regulatory Landscape
              *** Current Regulations
              *** Variations Across Countries
              *** Emerging Regulatory Challenges
              **** Ethical Considerations and Dual-Use Implications
              *** Future Directions for Regulation


              ** Case Studies
              *** Successful CRISPR Applications
              **** Notable Research Projects
              **** Clinical Trials
              *** Controversial Uses
              **** Germline Editing in Humans
              **** Agricultural Controversies


              ** Future of CRISPR Technology
              *** Innovations and Advancements
              *** Potential Impact on Society
              *** Balancing Benefits and Ethical Concerns
              *** Prospective Challenges and Opportunities

              ** Conclusion
              *** Summary of Key Points
              *** Final Thoughts on CRISPR Technology
              """


sections_list = sections.split('\n')
print(sections_list)





[' * CRISPR', '            ** Introduction', '            *** Definition of CRISPR', '            *** Historical Background', '            *** Importance in Genetic Research', '', '', '            ** CRISPR-Cas9 Mechanism', '            *** Overview of CRISPR-Cas9 Mechanism', '            *** Components of CRISPR-Cas9 System', '            **** CRISPR sequences', '            **** Cas proteins', '            *** Gene-editing Process', '            **** Key Steps in the CRISPR-Cas9 Process', '            **** Differences from Traditional Genetic Engineering Techniques', '            *** Enhancing Specificity of CRISPR-Cas9 System', '            **** Off-Target Effects and Their Mitigation', '            **** Improving Specificity of Genetic Alterations', '', '', '            ** Applications of CRISPR', '            *** Medical Applications', '            **** Gene Therapy', '            **** Treatment of Genetic Disorders', '            **** Risks in Human Therapeutic Applications', '  